## IncunabuLM

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
import math
import os.path

In [ ]:
print("Read parameters")
VOCAB_SIZE = 16384

# --- Training params
BATCH_SIZE = 8
BLOCK_SIZE = 2048
EVAL_INTERVAL = 250
EVAL_ITERS = 200
WEIGHT_DECAY = 0.1
CLIP_GRAD_NORM = 1.0
ACCUMULATION_STEPS = 16

# --- Learning rate scheduler
LEARNING_RATE = 3e-4
MIN_LEARNING_RATE = 1e-5
WARMUP_ITERS = 2000
MAX_ITERS = 50000

# --- Model params
N_EMBD = 768
N_HEAD = 12
N_LAYER = 12
DROPOUT = 0.2

# --- Fine tuning
MIN_LEARNING_RATE_FN = 5e-6
LEARNING_RATE_FN = 5e-5
WARMUP_ITERS_FN = 100
MAX_ITERS_FN = 3000

DATA_PATH = 'data_training.txt'
MODEL_PATH = 'best_model_50k_2048.pth'
TOKENIZER_PATH = 'bpe_tokenizer.json'
IMAGE_PATH = 'utilis/scribe.png'

BASE_MODEL_PATH = MODEL_PATH
FN_MODEL_PATH = 'fine_tune_model.pth'

SEED = 1337
torch.manual_seed(SEED)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

Read parameters


'\nwith open(DATA_PATH, \'r\', encoding=\'utf-8\') as f:\n    data = f.read()\n\nif os.path.isfile(TOKENIZER_PATH):\n    tokenizer = Tokenizer.from_file(TOKENIZER_PATH)\nelse:\n    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))\n    tokenizer.pre_tokenizer = ByteLevel()\n\n    trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=[\n        "[UNK]",\n        "[PAD]",\n        "[BOS]",\n        "[EOS]",\n        "<|endoftext|>"\n    ])\n\n    tokenizer.train([DATA_PATH], trainer)\n    tokenizer.decoder = ByteLevelDecoder()\n    tokenizer.save(TOKENIZER_PATH)\n\nencode = lambda s: tokenizer.encode(s).ids\ndecode = lambda l: tokenizer.decode(l)\n\ndata = torch.tensor(encode(data), dtype=torch.long)\n\nn = int(0.9*len(data))\ntrain_data = data[:n]\nval_data = data[n:]\n'

In [ ]:
# --- Init --- #
torch.set_float32_matmul_precision('high')

dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype) if DEVICE == 'cuda' else torch.amp.autocast(device_type='cpu')

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = f.read()

if os.path.isfile(TOKENIZER_PATH):
    tokenizer = Tokenizer.from_file(TOKENIZER_PATH)
else:
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = ByteLevel()

    trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=[
        "[UNK]",
        "[PAD]",
        "[BOS]",
        "[EOS]",
        "<|endoftext|>"
    ])

    tokenizer.train([DATA_PATH], trainer)
    tokenizer.decoder = ByteLevelDecoder()
    tokenizer.save(TOKENIZER_PATH)

In [ ]:
encode = lambda s: tokenizer.encode(s).ids
decode = lambda l: tokenizer.decode(l)

data = torch.tensor(encode(data), dtype=torch.long)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size, n_embd, block_size, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.size(-1)**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, n_embd, dropout, block_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super().__init__()
        hidden_dim = 4 * n_embd
        self.net = nn.Sequential(
            nn.Linear(n_embd, hidden_dim, bias=False),
            nn.SiLU(),
            nn.Linear(hidden_dim, n_embd, bias=False),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        output = self._norm(x.float()).type_as(x)
        return output * self.weight


class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout, block_size):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd, dropout, block_size)
        self.ffwd = FeedForward(n_embd, dropout)
        self.RMSN1 = RMSNorm(n_embd)
        self.RMSN2 = RMSNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.RMSN1(x))
        x = x + self.ffwd(self.RMSN2(x))
        return x


class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, n_embd, block_size, n_head, n_layer, dropout):
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embeddings_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, dropout=dropout, block_size=block_size) for _ in range(n_layer)])
        self.RMSN_f = RMSNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embeddings_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.RMSN_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None,
                 repetition_penalty=1.2, eos_token_id = None):
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]

            if repetition_penalty != 1.0:
                for i in range(idx_cond.shape[0]):
                    for token_id in set(idx_cond[i].tolist()):
                        if logits[i, token_id] > 0:
                            logits[i, token_id] /= repetition_penalty
                        else:
                            logits[i, token_id] *= repetition_penalty

            if temperature != 1.0:
                logits = logits / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)

            if eos_token_id is not None and idx_next.item() == eos_token_id:
                print("End of text")
                break

            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
def get_lr(it):
    #Linear warmup
    if it < WARMUP_ITERS:
        return LEARNING_RATE * it / WARMUP_ITERS
    if it > MAX_ITERS:
        return MIN_LEARNING_RATE

    decay_ratio = (it - WARMUP_ITERS) / (MAX_ITERS - WARMUP_ITERS)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return MIN_LEARNING_RATE + coeff * (LEARNING_RATE - MIN_LEARNING_RATE)

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)

    return x, y

In [ ]:
model = TransformerDecoder(VOCAB_SIZE, N_EMBD, BLOCK_SIZE, N_HEAD, N_LAYER, DROPOUT)
m = model.to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.amp.GradScaler(enabled=(dtype == 'float16'))

print("Compiling the model...")
model = torch.compile(model)
print("Model compiled.")

best_val_loss = float('inf')
best_model_path = 'best_model_cd.pth'

print(DEVICE)
try:
    print("Start training with gradient accumulation...")
    optimizer.zero_grad(set_to_none=True)

    for iter in range(MAX_ITERS):
        lr = get_lr(iter)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
            losses = estimate_loss()
            print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

            if losses['val'] < best_val_loss:
                best_val_loss = losses['val']
                # Poprawny sposób zapisu skompilowanego modelu
                torch.save(model._orig_mod.state_dict(), best_model_path)
                print(f"===> New best model saved to {best_model_path} with val_loss: {best_val_loss:.4f} <===")

        #Acc
        xb, yb = get_batch('train')
        with ctx:
            logits, loss = model(xb, yb)
            loss = loss / ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (iter + 1) % ACCUMULATION_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CLIP_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

except KeyboardInterrupt:
    print("Training interrupted by user. Saving final model...")
finally:
    final_model_path = MODEL_PATH if 'MODEL_PATH' in locals() else 'final_model.pth'
    torch.save(model._orig_mod.state_dict(), final_model_path)
    print(f"Final model saved to {final_model_path}")

Compiling the model...
Model compiled.
cuda
Start training with gradient accumulation...
Step 0: train loss 9.8741, val loss 9.8746
===> New best model saved to best_model_cd.pth with val_loss: 9.8746 <===
Step 250: train loss 8.8117, val loss 8.7916
===> New best model saved to best_model_cd.pth with val_loss: 8.7916 <===
Step 500: train loss 8.0556, val loss 8.0228
===> New best model saved to best_model_cd.pth with val_loss: 8.0228 <===
Step 750: train loss 7.5828, val loss 7.5442
===> New best model saved to best_model_cd.pth with val_loss: 7.5442 <===
Step 1000: train loss 7.4373, val loss 7.3893
===> New best model saved to best_model_cd.pth with val_loss: 7.3893 <===
Step 1250: train loss 7.2900, val loss 7.2553
===> New best model saved to best_model_cd.pth with val_loss: 7.2553 <===
Step 1500: train loss 7.1886, val loss 7.1360
===> New best model saved to best_model_cd.pth with val_loss: 7.1360 <===
Step 1750: train loss 7.0859, val loss 7.0439
===> New best model saved to be

In [ ]:
# Finetunning

with open('data_finetuning.txt', 'r', encoding='utf-8') as f:
    data = f.read()

tokenizer = Tokenizer.from_file(TOKENIZER_PATH)

encode = lambda s: tokenizer.encode(s).ids
decode = lambda l: tokenizer.decode(l)

data = torch.tensor(encode(data), dtype=torch.long)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
model = TransformerDecoder(VOCAB_SIZE, N_EMBD, BLOCK_SIZE, N_HEAD, N_LAYER, DROPOUT)
m = model.to(DEVICE)

# Load weights
if BASE_MODEL_PATH:
    print(f"Loading weights from base model: {BASE_MODEL_PATH}")
    state_dict = torch.load(BASE_MODEL_PATH, map_location=DEVICE)
    m.load_state_dict(state_dict, strict=False)
    print("Base model weights loaded successfully.")

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE_FN, weight_decay=WEIGHT_DECAY)
scaler = torch.amp.GradScaler(enabled=(dtype == 'float16'))

print("Compiling the model...")
model = torch.compile(model)
print("Model compiled.")

best_val_loss = float('inf')
best_model_path = 'fn_best_model.pth'

print(DEVICE)
try:
    print("Start training with gradient accumulation...")
    optimizer.zero_grad(set_to_none=True)

    for iter in range(MAX_ITERS_FN):
        lr = get_lr(iter)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
            losses = estimate_loss()
            print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

            if losses['val'] < best_val_loss:
                best_val_loss = losses['val']
                # Poprawny sposób zapisu skompilowanego modelu
                torch.save(model._orig_mod.state_dict(), best_model_path)
                print(f"===> New best model saved to {best_model_path} with val_loss: {best_val_loss:.4f} <===")

        #Acc
        xb, yb = get_batch('train')
        with ctx:
            logits, loss = model(xb, yb)
            loss = loss / ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (iter + 1) % ACCUMULATION_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CLIP_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

except KeyboardInterrupt:
    print("Training interrupted by user. Saving final model...")
finally:
    final_model_path = FN_MODEL_PATH if 'FN_MODEL_PATH' in locals() else 'fine_tune_model.pth'
    torch.save(model._orig_mod.state_dict(), final_model_path)
    print(f"Final model saved to {final_model_path}")

Loading weights from base model: best_model_50k_2048.pth
Base model weights loaded successfully.
Compiling the model...
Model compiled.
cuda
Start training with gradient accumulation...
Step 0: train loss 5.2570, val loss 5.0952
===> New best model saved to fn_best_model.pth with val_loss: 5.0952 <===
Step 250: train loss 4.6169, val loss 4.6543
===> New best model saved to fn_best_model.pth with val_loss: 4.6543 <===
Step 500: train loss 4.4465, val loss 4.5825
===> New best model saved to fn_best_model.pth with val_loss: 4.5825 <===
Step 750: train loss 4.3531, val loss 4.5511
===> New best model saved to fn_best_model.pth with val_loss: 4.5511 <===
Step 1000: train loss 4.2729, val loss 4.5212
===> New best model saved to fn_best_model.pth with val_loss: 4.5212 <===
Step 1250: train loss 4.1961, val loss 4.5031
===> New best model saved to fn_best_model.pth with val_loss: 4.5031 <===
Step 1500: train loss 4.1501, val loss 4.4998
===> New best model saved to fn_best_model.pth with va